# Structured output parser

이 출력 파서는 여러 필드를 반환하고자 할 때 사용할 수 있습니다. Pydantic/JSON 파서가 더 강력하지만, 이는 덜 강력한 모델에 유용합니다.


In [1]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.


In [7]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 출처, 웹사이트 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

우리는 이제 응답이 어떻게 포맷되어야 하는지에 대한 지시사항이 포함된 문자열을 받게 되며, 그것을 우리의 프롬프트에 삽입합니다.


- `output_parser.get_format_instructions()` 함수를 호출하여 포맷 지시사항을 가져옵니다.
- `PromptTemplate` 클래스를 사용하여 사용자의 질문에 최대한 답변하는 프롬프트 템플릿을 생성합니다.
- 템플릿에는 `question`이라는 입력 변수와 `format_instructions`라는 부분 변수가 포함됩니다.


In [8]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [4]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [9]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

- `chain.stream` 메소드를 사용하여 "세종대왕의 업적은 무엇인가요?" 라는 질문에 대한 스트림을 반복 처리합니다.
- 반복 과정에서 스트림의 결과를 출력합니다.


In [10]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화 발전에 큰 기여를 한 대표적 인물입니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
